# Import libraries

In [8]:
import pandas as pd
import numpy as np

# Load data

In [2]:
# Load the CSV file
df = pd.read_csv("./Stock_Data/stocks.csv") 
df

,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,AAPL,2023-02-07,150.639999,155.229996,150.639999,154.649994,154.414230,83322600
1,AAPL,2023-02-08,153.880005,154.580002,151.169998,151.919998,151.688400,64120100
2,AAPL,2023-02-09,153.779999,154.330002,150.419998,150.869995,150.639999,56007100
3,AAPL,2023-02-10,149.460007,151.339996,149.220001,151.009995,151.009995,57450700
4,AAPL,2023-02-13,150.949997,154.259995,150.919998,153.850006,153.850006,62199000
...,...,...,...,...,...,...,...,...
243,GOOG,2023-05-01,107.720001,108.680000,107.500000,107.709999,107.709999,20926300
244,GOOG,2023-05-02,107.660004,107.730003,104.500000,105.980003,105.980003,20343100
245,GOOG,2023-05-03,106.220001,108.129997,105.620003,106.120003,106.120003,17116300
246,GOOG,2023-05-04,106.160004,106.300003,104.699997,105.209999,105.209999,19780600


In [3]:
df.columns

Index(['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close',
       'Volume'],
      dtype='object')

# 1. Handle Missing Values

In [4]:
if df.isnull().sum().sum() > 0:
    print("Missing values found. Handling missing values...")
    for col in ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']:
        if df[col].isnull().any():
            if pd.api.types.is_numeric_dtype(df[col]):
                df[col].fillna(df[col].median(), inplace=True)
            else:
                print(f"Column {col} is not numeric and contains missing values.")
                df = df.dropna(subset=[col])
    if df['Date'].isnull().any():
        df['Date'] = df['Date'].ffill().bfill()

else:
    print("No missing values found.")

No missing values found.


# 2. Data Type Correction

In [5]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
numerical_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].fillna(0)

# 3. Remove Duplicates

In [6]:
df.drop_duplicates(inplace=True)

# 4. Handle Outliers

In [9]:
def remove_outliers(df, col, threshold=3):
    if pd.api.types.is_numeric_dtype(df[col]):
        z_scores = np.abs((df[col] - df[col].mean()) / df[col].std())
        df = df[z_scores < threshold]
    return df

for col in ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']:
    df = remove_outliers(df, col)

# 5. Sort by Date

In [10]:
df.sort_values(by='Date', inplace=True)
df.reset_index(drop=True, inplace=True)

# Report of results

In [11]:
print(df.head())
print(df.dtypes)
print(df.isnull().sum())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-02-07  150.639999  155.229996  150.639999  154.649994   
1   GOOG 2023-02-07  103.629997  108.669998  103.547997  108.040001   
2   MSFT 2023-02-07  260.529999  268.769989  260.079987  267.559998   
3   NFLX 2023-02-07  358.510010  364.179993  354.179993  362.950012   
4   GOOG 2023-02-08  102.690002  103.580002   98.455002  100.000000   

    Adj Close    Volume  
0  154.414230  83322600  
1  108.040001  33738800  
2  266.891510  50841400  
3  362.950012   6289400  
4  100.000000  73546000  
Ticker               object
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object
Ticker       0
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


# Save the clean data

In [12]:
df.to_csv("./Stock_Data/clean_stocks.csv", index=False)